
# 🔐 Cracking Enigma the Way Turing Did
## Bayesian Inference, Log-Odds, and the Bombe — in Python

This notebook upgrades our Enigma simulation to be **historically faithful**:

✅ Log-odds scoring (Turing's Banburismus idea)  
✅ Multiple intercepted messages (sequential Bayes)  
✅ English language likelihood model  
✅ Live animation of belief collapse  

This is *exactly* how probability defeated brute force.



## 1. Enigma Machine (Rotors + Reflector)


In [ ]:
import string
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

: 

In [ ]:

import string
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

ALPHABET = string.ascii_uppercase
A2I = {c:i for i,c in enumerate(ALPHABET)}
I2A = {i:c for i,c in enumerate(ALPHABET)}

ROTOR_I   = "EKMFLGDQVZNTOWYHXUSPAIBRCJ"
ROTOR_II  = "AJDKSIRUXBLHWTMCQGZNPYFVOE"
ROTOR_III = "BDFHJLCPRTXVZNYEIWGAKMUSQO"
REFLECTOR = "YRUHQSLDPXNGOKMIEBFZCWVJAT"

rotors = [ROTOR_I, ROTOR_II, ROTOR_III]

def enigma_encrypt(text, positions):
    pos = positions.copy()
    out = ""
    
    for ch in text.upper():
        if ch not in ALPHABET:
            out += ch
            continue
        
        pos[0] = (pos[0] + 1) % 26
        if pos[0] == 0:
            pos[1] = (pos[1] + 1) % 26
        if pos[1] == 0:
            pos[2] = (pos[2] + 1) % 26
        
        c = A2I[ch]
        for i in range(3):
            c = (A2I[rotors[i][(c + pos[i]) % 26]] - pos[i]) % 26
        
        c = A2I[REFLECTOR[c]]
        
        for i in reversed(range(3)):
            c = (rotors[i].index(I2A[(c + pos[i]) % 26]) - pos[i]) % 26
        
        out += I2A[c]
    
    return out



## 2. English Language Model (Likelihood)

Instead of keyword matching, we score decrypted text using
**English letter frequencies**.


In [ ]:

english_freq = {
    'E':0.127, 'T':0.091, 'A':0.082, 'O':0.075, 'I':0.070, 'N':0.067,
    'R':0.060, 'S':0.063, 'H':0.061, 'L':0.040, 'D':0.043, 'C':0.028,
    'U':0.028, 'M':0.024, 'F':0.022, 'Y':0.020, 'W':0.024, 'G':0.020,
    'P':0.019, 'B':0.015, 'V':0.010, 'K':0.008, 'X':0.002,
    'Q':0.001, 'J':0.002, 'Z':0.001
}

def english_log_likelihood(text):
    ll = 0.0
    for ch in text:
        if ch in english_freq:
            ll += np.log(english_freq[ch])
    return ll



## 3. Intercepted Messages (Sequential Bayes)


In [ ]:

true_key = [3, 14, 7]

messages = [
    "WEATHER REPORT FROM ATLANTIC",
    "WEATHER CONDITIONS STABLE",
    "REPORT SENT AT DAWN"
]

ciphertexts = [enigma_encrypt(m, true_key) for m in messages]
ciphertexts



## 4. Hypothesis Space & Log-Odds (Turing Score)

We maintain **log-posteriors** instead of probabilities.
This avoids underflow and mirrors Turing's scoring system.


In [ ]:

keys = [(a,b,c) for a in range(6) for b in range(6) for c in range(6)]
log_post = np.zeros(len(keys))  # uniform prior in log-space



## 5. Sequential Bayesian Updates


In [ ]:

history = []

for ct in ciphertexts:
    for i, key in enumerate(keys):
        decrypted = enigma_encrypt(ct, list(key))
        log_post[i] += english_log_likelihood(decrypted)
    
    # normalize for numerical stability
    log_post -= np.max(log_post)
    probs = np.exp(log_post)
    probs /= probs.sum()
    history.append(probs.copy())



## 6. Final Result


In [ ]:

best_idx = np.argmax(history[-1])
best_key = keys[best_idx]

print("Recovered key:", best_key)
print("Decrypted final message:")
print(enigma_encrypt(ciphertexts[-1], list(best_key)))



## 7. Live Animation — The Bombe in Action


In [ ]:

fig, ax = plt.subplots()
bars = ax.bar(range(len(keys)), history[0])
ax.set_ylim(0, max(history[-1])*1.1)
ax.set_title("Bayesian Belief Over Enigma Keys")

def update(frame):
    for bar, p in zip(bars, history[frame]):
        bar.set_height(p)
    ax.set_xlabel(f"Intercept {frame+1}")
    return bars

ani = FuncAnimation(fig, update, frames=len(history), interval=1200)
plt.show()
